# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [7]:
# Load the dotenv extension
%load_ext dotenv

# Load environment variables from .env
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob

# Load the environment variable PRICE_DATA
ft_dir = os.getenv('PRICE_DATA')

# If recursive is true, the pattern “**” will match any files and zero or more directories, subdirectories and symbolic links to directories.
parquet_files = glob(ft_dir + '**/*.parquet', recursive=True)
parquet_files

['../../05_src/data/prices\\AAPL\\AAPL_2000\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2000\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.1.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.0.parquet',
 '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.1.parquet',
 '../../

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Reading multiple parquet files into a Dask DataFrame
df = dd.read_parquet(parquet_files)

# Grouping by Ticker column and group_keys = False ensures that the group labels do not appear in the index of the result. 
# .apply() will apply the given function to each group. 
# lambda is the other way of defining a function. E.g .shift(1) will take the Close column value from the previous row and assign it to the new column name Close_lag_1.

df_lags = df.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),
                       Close_lag_2 = x['Adj Close'].shift(1))
)

dd_feat = df_lags.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(returns = (x['Close'] / x['Close_lag_1']) - 1,
                       hi_lo_range = x['High'] - x['Low'])
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [11]:
# Converting Dask data frame to a pandas data frame. Further calculating moving average using a window of 10 days. First nine values will be NaN as there are not 10 values for the mean.
dd_feat = dd_feat.compute()
dd_feat['rolling_mean'] = dd_feat['returns'].rolling(10).mean()
dd_feat

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Close_lag_2,returns,hi_lo_range,rolling_mean
144864,2009-01-02 00:00:00+00:00,JNPR,14.243329,18.389999,18.469999,17.350000,17.350000,5909400.0,2009,NaN,NaN,NaN,1.119999,NaN
144928,2009-01-05 00:00:00+00:00,JNPR,14.320775,18.490000,18.650000,18.100000,18.330000,6350800.0,2009,18.389999,14.243329,0.005438,0.549999,NaN
144992,2009-01-06 00:00:00+00:00,JNPR,14.537646,18.770000,18.840000,18.180000,18.540001,11441400.0,2009,18.490000,14.320775,0.015143,0.660000,NaN
145056,2009-01-07 00:00:00+00:00,JNPR,13.801848,17.820000,18.830000,17.629999,18.639999,10060800.0,2009,18.770000,14.537646,-0.050613,1.200001,NaN
145120,2009-01-08 00:00:00+00:00,JNPR,14.088423,18.190001,18.280001,17.549999,17.780001,9362000.0,2009,17.820000,13.801848,0.020763,0.730001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386090,2023-12-22 00:00:00+00:00,NTAP,87.344612,89.400002,89.989998,89.010002,89.930000,828400.0,2023,89.550003,87.491165,-0.001675,0.979996,0.000069
386154,2023-12-26 00:00:00+00:00,NTAP,87.325066,89.379997,89.660004,89.180000,89.360001,525400.0,2023,89.400002,87.344612,-0.000224,0.480003,-0.000356
386218,2023-12-27 00:00:00+00:00,NTAP,86.563004,88.599998,89.669998,88.419998,89.360001,726800.0,2023,89.379997,87.325066,-0.008727,1.250000,0.000498
386282,2023-12-28 00:00:00+00:00,NTAP,86.181969,88.209999,88.760002,88.029999,88.580002,962700.0,2023,88.599998,86.563004,-0.004402,0.730003,-0.001461


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

+ It's not necessary to convert to pandas to calculate the moving average return. We can also work with the csv files. Also, repartition() can be used with map_overlap() without converting Dask DataFrame to pandas for calculating the rolling average.  
Dask DataFrame is lazy, meaning it doesn't compute results immediately. It does not show actual data; it just provides an overview. So, the other method is to use .compute() to show actual results to calculate the rolling average, and the compute method converts the dask DataFrame to pandas DataFrame. For data that fits into RAM, pandas can often be faster and easier to use than Dask DataFrame.  

+ It depends on the size of the dataset. If it's small, pandas is the better than Dask. However, if it's large dataset, Dask is the best choice. Our dataset is not very large, so converting to pandas is good choice.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.